In [1]:
import os
import numpy as np
import pandas as pd
from numba import njit
import open3d as o3d
import networkx as nx
import functools
import time

In [2]:
def read_dat_sylinder( fname):
    # Read a SylinderAscii_X.dat file

    # open the file and read the lines
    with open(fname, 'r') as file1:
        filecontent = file1.readlines()

        # Initialize numpy arrays for data
        gids = np.zeros( len(filecontent)-2, dtype=int)
        rad = np.zeros(len(filecontent)-2)
        pos0 = np.zeros( (len(filecontent)-2,3))
        pos1 = np.zeros( (len(filecontent)-2,3))
        ort = np.zeros( (len(filecontent)-2,3))

        # Delete the first two lines because they dont have any data
        filecontent[0:2] = []
        for idx,line in enumerate(filecontent):

            # Split the string with space-delimited and convert strings into useful data types
            data = line.split()
            gids[idx] = int(data[1])

            dat = np.array( list(map(float,data[2::])) )
            rad[idx] = dat[0]
            pos0[idx,:] = dat[1:4]
            pos1[idx,:] = dat[4::]
            xi = pos1[idx,:] - pos0[idx,:]
            ort[idx,:] =  xi/np.sqrt(xi.dot(xi))

    # Store data as a dataframe
    df = pd.DataFrame({
        'gid': gids,
        'radius' : rad,
        'pos0' : list(pos0),
        'pos1': list(pos1),
        'orientation': list(ort)
        })
    return df

def read_dat_protein( fname):
    # Read a ProetinAscii_X.dat file

    # open the file and read the lines
    with open(fname, 'r') as file1:
        filecontent = file1.readlines()

        # Delete the first two lines because they dont have any data
        filecontent[0:2] = []

        # Initialize numpy arrays for data
        gids = np.zeros( len(filecontent), dtype=int)
        pos0 = np.zeros( (len(filecontent),3))
        pos1 = np.zeros( (len(filecontent),3))
        link0 = np.zeros( len(filecontent), dtype=int)
        link1 = np.zeros( len(filecontent), dtype=int)

        for idx,line in enumerate(filecontent):

            # Split the string with space-delimited and convert strings into useful data types
            data = line.split()
            # pdb.set_trace()
            gids[idx] = int(data[1])
            link0[idx] = int(data[9])
            link1[idx] = int(data[10])
            dat = np.array( list(map(float,data[2:9])) )
            pos0[idx,:] = dat[1:4]
            pos1[idx,:] = dat[4::]

    # Store data as a dataframe
    df = pd.DataFrame({
        'gid': gids,
        'pos0' : list(pos0),
        'pos1': list(pos1),
        'link0': link0,
        'link1': link1
        })
    return df

In [22]:
def timer( func):
    """ Print runtime of decorated functions """
    @functools.wraps( func)
    def wrapper_timer( *args, **kwargs):
        start_time = time.perf_counter()
        value = func( *args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def calc_nematic_order(orient_array):
    # Calculates the maximum eigenvalue of the nematic tensor Q

    # calculate Q tensor
    Q = calc_nematic_tensor(orient_array)
    S = np.sqrt(np.tensordot(Q, Q)*1.5)
    return S

@timer
@njit
def calc_polar_order(orient_array):
    # Calculates the polar order given the array of filament orientations

    # Initialize P vector
    Pvec = np.zeros(3)

    # Take a mean of all orientations
    for irow in np.arange(orient_array.shape[0]):
        Pvec += orient_array[irow,:]
    Pvec = Pvec / orient_array.shape[0]
    P = np.linalg.norm( Pvec)
    return P

@njit
def calc_nematic_tensor( orient_array):
    # Calculates the nematic tensor Q

    # initialize Q tensor
    Q = np.zeros((3,3))
    # sum over all filaments, taking their outer products
    for irow in np.arange(orient_array.shape[0]):
        Q += np.outer( orient_array[irow,:], orient_array[irow,:])

    # complete mean calculation by dividing by number of filaments and subtracting identity.
    Q = Q/orient_array.shape[0] - np.identity(3)/3
    return Q


@timer
def get_largest_cc( nodes, edge0, edge1):
    # Get largesst connected component
    # nodes is a list of integers representing node indices
    # edge0 / edge1 are the list of nodes that each edge connects to.
    # Indices whose values are -1 are ignored in both edge0 and edge1.

    # Create a graph for filaments
    g = nx.Graph()
    g.add_nodes_from( nodes)

    # add edges to the graph
    for e0,e1 in zip( edge0,edge1):
        if e0 != -1 and e1 != -1:
            g.add_edge(e0, e1)

    # find connected component largest
    cc = list( max(nx.connected_components(g), key=len) )

    # also get a boolean array representing nodes that are in the largest cc
    cc_bool = np.zeros(len(nodes), dtype=bool)
    cc_bool[ cc] = True
    return cc, cc_bool

@timer
@njit
def get_edges_in_largest_cc( cc, edge0, edge1):

    # initialize
    idx_edges = []
    n_edges = 0

    # num xlinks in biggest cluster
    for idx in np.arange(len(edge0)):
        if edge0[idx] != -1 or edge1[idx] != -1:
            idx_edges.append( idx_edges)
            n_edges+=1

    return idx_edges, n_edges



@njit
def calc_distance_pbc(p0,p1,boxsize):
    # distance between two points in the nearest image convention
    # can use multidimensional arrays for distances between multiple points
    dist = np.absolute( p1-p0)
    for idx in np.arange(dist.shape[-1]):
        if len(dist.shape) == 1:
            k = np.floor( dist[idx]/(0.5*boxsize[idx]))
            dist[idx] -= k*boxsize[idx]
        elif len(dist.shape) == 2:
            k = np.floor( dist[:,idx]/(0.5*boxsize[idx]))
            dist[:,idx] -= k*boxsize[idx]
        # elif len(dist.shape) == 3:
            # k = np.floor( dist[:,:,idx]/(0.5*boxsize[idx]))
            # dist[:,:,idx] -= k*boxsize[idx]
    return np.absolute(dist)

@njit
def calc_mean_pbc(p0,p1,boxsize):
    # mean of the two points in the nearest image
    dist = np.absolute(p1-p0)
    for idx in np.arange(dist.shape[-1]):
        if len(dist.shape) == 1:
            k = np.floor( dist[idx]/(0.5*boxsize[idx]))
            p1[idx] -= k*boxsize[idx]
        elif len(dist.shape) == 2:
            k = np.floor( dist[:,idx]/(0.5*boxsize[idx]))
            p1[:,idx] -= k*boxsize[idx]
        # elif len(dist.shape) == 3:
            # k = np.floor( dist[:,:,idx]/(0.5*boxsize[idx]))
            # p1[:,:,idx] -= k*boxsize[idx]
    return (p0 + p1)/2

@timer
@njit
def calc_mean_separation(coords,boxsize):
    # Calculate mean pair-pair separation

    # Initialize
    num_coords = coords.shape[0]
    mu = 0
    sig = 0

    mean_p2p = calc_mean_p2p_dist( coords, boxsize)
    for idx in np.arange( num_coords):
        mu += mean_p2p[idx]
    mu = mu/num_coords
    sig = np.std( mean_p2p)
    return mu,sig


@njit
def calc_mean_p2p_dist(coords, boxsize):
    # Calculate mean separation of every filament from all other pairs

    # Initialize
    num_coords = coords.shape[0]
    mean_p2p = np.zeros( num_coords)

    # Calculate p2p distance matrix
    p2p_mat = calc_p2p_dist_mat(coords, boxsize)
    for idx in np.arange( num_coords):
        for idx2 in np.arange(num_coords):
            mean_p2p[idx] += p2p_mat[idx,idx2]
        mean_p2p[idx] = mean_p2p[idx]/num_coords

    return mean_p2p

@njit
def calc_p2p_dist_mat(coords, boxsize):

    # Initialize
    num_coords = coords.shape[0]
    p2p = np.zeros( (num_coords, num_coords))

    # Loop over reference filament. For ref filament,
    for jf in np.arange( num_coords):

        dist_sum = 0
        dist = np.absolute( coords[jf,:]-coords)

        # get distance to all other filaments
        for jf2 in np.arange( num_coords):

            # Skip if same as the reference filament
            if jf==jf2:
                continue
            # If distance with j1 and j2 switched already calculated, copy from matrix
            elif p2p[jf2,jf] != 0:
                p2p[jf,jf2] = p2p[jf2,jf]
                dist_sum += p2p[jf2,jf]
            # Otherwise, calculate distance
            else:
                # We need to find the norm distance. We will code up a manual
                # summation of squares and then take a square root. This is
                # faster with njit.
                # We apply box-size considerations to find the distance to the closest image
                dist_xyz = 0
                # get distance
                for idx in np.arange( len(boxsize)):
                    k = np.floor( dist[jf2,idx]/(0.5*boxsize[idx]))
                    dist[jf2,idx] -= k*boxsize[idx]
                    dist_xyz += dist[jf2,idx]**2
                p2p[jf,jf2] = np.sqrt( dist_xyz)
                dist_sum += p2p[jf,jf2]
    return p2p

@njit
def msd_pairs(lags,c,nT,nF,boxsize,boolRodsInside):
    mu = np.zeros(lags.shape)
    sig = np.zeros(lags.shape)
    for idx in range(lags.size):
        pptdiff = np.zeros(nT-lags[idx])
        for it in range(pptdiff.size):

            #find rods inside at both times
            rods_in = np.logical_and( boolRodsInside[it],boolRodsInside[it+lags[idx]])

            ppt1=dist2all(c[it,:,:],nT,nF,boxsize,rods_in)
            ppt2=dist2all(c[it+lags[idx],:,:],nT,nF,boxsize,rods_in)

            res = 0
            for idx2 in range(ppt1.size):
                res+=(ppt2[idx2]-ppt1[idx2])**2
            pptdiff[it] = res/ppt1.size
        mu[idx] = np.mean(pptdiff)
        sig[idx] = np.std(pptdiff)
    return mu,sig



@timer
@njit
def calc_protein_energy( lengths, rest_length):

    # Energy
    sum_energy = 0
    for ix in np.arange( len(lengths) ):
        sum_energy += 0.5*( lengths[ix]-rest_length)**2
    xlink_mu = sum_energy / lengths.shape[0]

    sum_energy_sq = 0
    for ix in np.arange( len(lengths) ):
        sum_energy_sq += ( 0.5*( lengths[ix]-rest_length)**2 - xlink_mu)
    xlink_std = np.sqrt( sum_energy_sq/lengths.shape[0])

    return xlink_mu, xlink_std


@timer
def calc_aspect_ratio( coords):
    # Calculate aspect ratio of bounding box of a collection of 3D points

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector( coords)
    obb = pcd.get_oriented_bounding_box()

    xx = obb.extent
    aspect_ratio = max( xx) / np.mean(sorted(xx)[0:2])
    return aspect_ratio

@timer
@njit
def calc_z_ordering( orient_array):
    # Calculate alignment in z

    z_axis = np.array( [0.,0.,1.])
    sum_all = 0
    for idx in np.arange( orient_array.shape[0]):
        sum_all += np.absolute( orient_array[idx,:].dot(z_axis) )
    return sum_all/orient_array.shape[0]


@njit
def pair_partition_func_centers(centers):
    
    q = np.zeros((centers.shape[0], centers.shape[0]))
    # for each pair of centers, calculate pair_partition function
    for idx1 in np.arange( centers.shape[0]):
        q[idx1,:] = pair_partition_func_i_centers( 
            centers[idx1,:], centers, q[idx1,:])
    return q

@njit
def pair_partition_func_i_centers(r0,r1,q):
    
    # distance between centers
    dist = r0-r1
    for idx in np.arange( dist.shape[1]):
        q+= dist[:,idx]**2
    return q


@timer
@njit
def calc_local_polar_order( c, orients, boxsize):
    # Calculate the local polar order
    
    porder_par = 0
    porder_apar = 0
    
    # Define a maximum distance for infinite square potential well
    dist_max = 4*0.025
    
    # coordinate indices within the distance range
    q = pair_partition_func_centers(c)

    for idx in np.arange( q.shape[0]):
        
        o_parallel = np.zeros(3)
        o_aparallel = np.zeros(3)
        n_parallel = 0
        n_aparallel = 0
        
        # orientation of reference filament
        o1 = orients[idx,:]
        for idx1 in np.arange(q.shape[1]):
            
            if o1.dot( orients[idx1,:]) > 0:
                o_parallel += q[idx,idx1]*orients[idx1,:]
                n_parallel += 1
            else:
                o_aparallel += q[idx,idx1]*orients[idx1,:]
                n_aparallel += 1
        
        porder_par += np.linalg.norm( o_parallel/n_parallel)
        porder_apar += np.linalg.norm( o_aparallel/n_aparallel)

    return porder_par/c.shape[0], porder_apar/c.shape[0]

In [30]:
# fpath = "/Users/saadjansari/Documents/Projects/AMSOS/resultsSummit/Tactoids/scan_filamin_6400/run/f1"
fpath = "/Users/saadjansari/Documents/Projects/AMSOS/resultsSummit/Confinement/scan_d_pf_const_num/run_1/pf02_d10/s0"

fpath_actin = os.path.join(fpath, "result/result0-399/SylinderAscii_150.dat")
fpath_xlinks = os.path.join(fpath, "result/result0-399/ProteinAscii_150.dat")

df_sylinder = read_dat_sylinder(fpath_actin)
df_protein = read_dat_protein(fpath_xlinks)

In [31]:
import time
t0 = time.time()
boxsize=np.array([3,3,3])
data={}
# Filament centers
c = calc_mean_pbc(
    np.array( df_sylinder.pos0.tolist()),
    np.array( df_sylinder.pos1.tolist()),
    boxsize)

# Check existence of a cluster
# Get largest connected component using information about xlinks
cc, cc_bool = get_largest_cc(
        df_sylinder.gid.tolist(),
        df_protein.link0.tolist(),
        df_protein.link1.tolist() )
df_sylinder_cc = df_sylinder[cc_bool]


# Global order
# Bulk and Cluster
data['S_bulk'] = calc_nematic_order( np.array( df_sylinder.orientation.tolist()) )
data['P_bulk'] = calc_polar_order( np.array( df_sylinder.orientation.tolist()) )
data['S_cluster'] = calc_nematic_order( np.array( df_sylinder_cc.orientation.tolist()) )
data['P_cluster'] = calc_polar_order( np.array( df_sylinder_cc.orientation.tolist()) )

# Pair-pair separation

# Filament centers
c = calc_mean_pbc(
    np.array( df_sylinder.pos0.tolist()),
    np.array( df_sylinder.pos1.tolist()),
    boxsize)
# data['mean_sep_bulk'] = calc_mean_separation( c, boxsize)
data['mean_sep_cluster'] = list(calc_mean_separation( c[cc,:], boxsize))

# Xlink
# Energy
# xlink lengths
xlink_lengths = np.linalg.norm(
        calc_distance_pbc(
            np.array( df_protein.pos0.tolist()),
            np.array( df_protein.pos1.tolist()),
            boxsize), axis=1)
data['xlink_energy'] = list(calc_protein_energy( xlink_lengths, 0.05))

#data['tactoid_aspect_ratio'] = calc_aspect_ratio( c[cc,:])
data['z_order'] = calc_z_ordering( np.array( df_sylinder.orientation.tolist()) )
data['xy_order'] = calc_xy_ordering( np.array( df_sylinder.orientation.tolist()) )
a,b = calc_local_polar_order( c, np.array( df_sylinder.orientation.tolist()),boxsize)

t1 = time.time()
print('Time elapsed = {}'.format(t1-t0))

Finished 'get_largest_cc' in 0.0011 secs
Finished 'calc_nematic_order' in 0.0001 secs
Finished 'calc_polar_order' in 0.0000 secs
Finished 'calc_nematic_order' in 0.0001 secs
Finished 'calc_polar_order' in 0.0000 secs
Finished 'calc_mean_separation' in 0.0001 secs
Finished 'calc_protein_energy' in 0.0000 secs
Finished 'calc_z_ordering' in 0.0000 secs
Finished 'calc_xy_ordering' in 0.0000 secs
Finished 'calc_local_polar_order' in 0.0014 secs
Time elapsed = 0.011438131332397461


In [57]:
r0 = df_sylinder.pos0[0]
u0 = df_sylinder.orientation[0]
r1 = df_sylinder.pos0[1]
u1 = df_sylinder.orientation[1]

@njit
def pair_partition_func_centers(centers):
    
    q = np.zeros((centers.shape[0], centers.shape[0]))
    # for each pair of centers, calculate pair_partition function
    for idx1 in np.arange( centers.shape[0]):
        q[idx1,:] = pair_partition_func_i_centers( 
            centers[idx1,:], centers, q[idx1,:])
    return q

@njit
def pair_partition_func_i_centers(r0,r1,q):
    
    # distance between centers
    dist = r0-r1
    for idx in np.arange( dist.shape[1]):
        q+= dist[:,idx]**2
    return q

t0 = time.time()
q = pair_partition_func_centers(
    np.array( df_sylinder.pos0.tolist() ))
print('time elapsed = {}'.format(time.time()-t0))

time elapsed = 1.5853219032287598


In [10]:
 # Define a maximum distance for infinite square potential well
dist_max = 4*0.025

# coordinate indices within the distance range
c_within = ( calc_p2p_dist_mat( np.array( df_sylinder.pos0.tolist() ), boxsize) < dist_max )

In [33]:
b

8.403236936566367

In [ ]:
a0 + np.linspace(0,1,5)[None]*(a1-a0)

In [ ]:
np.linspace(0,1,5)[None]

In [41]:
@njit
def f1(n):
    
    q = np.zeros((n, n))
    # for each pair of centers, calculate pair_partition function
    for idx1 in np.arange( n):
        aa = f2(n)
    return q

@njit
def f2(n):
    idx=0
    for idx in np.arange(n):
        idx+=1
    return idx

t0 = time.time()
q = f1(6000)
print('time elapsed = {}'.format(time.time()-t0))

time elapsed = 0.3822059631347656


In [55]:
cc

[8,
 10,
 12,
 16,
 20,
 21,
 22,
 23,
 25,
 26,
 27,
 28,
 31,
 32,
 33,
 34,
 37,
 40,
 41,
 42,
 44,
 46,
 48,
 50,
 53,
 56,
 57,
 59,
 70,
 72,
 75,
 80,
 86,
 88,
 89,
 92,
 93,
 95,
 98,
 99,
 102,
 104,
 105,
 106,
 107,
 110,
 111,
 112,
 113,
 114,
 115,
 119,
 121,
 122,
 123,
 124,
 125,
 127,
 129,
 131,
 134,
 135,
 136,
 142,
 144,
 146,
 147,
 151,
 153,
 154,
 155,
 157,
 160,
 161,
 165,
 166,
 167,
 169,
 171,
 172,
 174,
 175,
 182,
 188,
 192,
 194,
 196,
 197,
 206,
 208,
 209,
 219,
 221,
 226,
 227,
 228,
 229,
 232,
 236,
 238,
 239,
 240,
 241,
 244,
 252,
 253,
 255,
 257,
 259,
 261,
 262,
 265,
 267,
 268,
 269,
 270,
 271,
 274,
 278,
 279,
 285,
 286,
 287,
 290,
 291,
 292,
 293,
 296,
 297,
 299,
 300,
 301,
 303,
 306,
 307,
 308,
 309,
 312,
 314,
 315,
 317,
 319,
 320,
 321,
 323,
 324,
 329,
 330,
 332,
 336,
 338,
 339,
 342,
 346,
 347,
 349,
 350,
 352,
 355,
 356,
 359,
 360,
 361,
 363,
 366,
 367,
 368,
 369,
 371,
 372,
 375,
 377,
 378,
 37

In [3]:
import progressbar
from time import sleep
bar = progressbar.ProgressBar(maxval=20,
    widgets=[progressbar.Percentage(), ' ', progressbar.Bar('=', '[', ']'), ' ', progressbar.Timer()])
bar.start()
for i in range(20):
    bar.update(i+1)
    sleep(0.1)
bar.finish()

100% [==================================================] Elapsed Time: 0:00:02


In [77]:
import sys
sys.stdout.flush()

TypeError: 'property' object is not callable

In [5]:
val = input('give me a number: ', end='\r')
print('')

TypeError: raw_input() got an unexpected keyword argument 'end'

In [84]:
import progressbar 
import time 
  
def animated_marker(): 
      
    widgets = ['Loading: ', progressbar.AnimatedMarker()] 
    bar = progressbar.ProgressBar(maxval=20,widgets=widgets).start() 
      
    for i in range(50): 
        time.sleep(0.1) 
        bar.update(i) 
          
# Driver's code 
animated_marker() 

ValueError: Value out of range

In [77]:
df = pd.DataFrame()
df = df.append(data, ignore_index=True)
df = df.append(data, ignore_index=True)
df = df.append(data, ignore_index=True)
df = df.append(data, ignore_index=True)
df = df.append(data, ignore_index=True)

df.head()
# df.to_feather('/Users/saadjansari/Desktop/temp.feather')
# df2 = pd.read_feather('/Users/saadjansari/Desktop/temp.feather')

,P_bulk,P_cluster,S_bulk,S_cluster,mean_sep_cluster,xlink_energy,xy_order,z_order
0,0.130446,0.126716,0.253305,0.246278,"[0.34413641697471997, 0.06424119986320587]","[0.0004060710402280934, 1.3535512156251837e-10]",0.602037,0.377536
1,0.130446,0.126716,0.253305,0.246278,"[0.34413641697471997, 0.06424119986320587]","[0.0004060710402280934, 1.3535512156251837e-10]",0.602037,0.377536
2,0.130446,0.126716,0.253305,0.246278,"[0.34413641697471997, 0.06424119986320587]","[0.0004060710402280934, 1.3535512156251837e-10]",0.602037,0.377536
3,0.130446,0.126716,0.253305,0.246278,"[0.34413641697471997, 0.06424119986320587]","[0.0004060710402280934, 1.3535512156251837e-10]",0.602037,0.377536
4,0.130446,0.126716,0.253305,0.246278,"[0.34413641697471997, 0.06424119986320587]","[0.0004060710402280934, 1.3535512156251837e-10]",0.602037,0.377536


In [44]:
df2

,P_bulk,P_cluster,S_bulk,S_cluster,mean_sep_cluster,tactoid_aspect_ratio,xlink_energy
0,1.0,1.0,1.0,1.0,"[0.5253747094739183, 0.016854412839272838]",1.008001,"[0.00012320532641838186, nan]"
1,1.0,1.0,1.0,1.0,"[0.5253747094739183, 0.016854412839272838]",1.008001,"[0.00012320532641838186, nan]"
2,1.0,1.0,1.0,1.0,"[0.5253747094739183, 0.016854412839272838]",1.008001,"[0.00012320532641838186, nan]"
3,1.0,1.0,1.0,1.0,"[0.5253747094739183, 0.016854412839272838]",1.008001,"[0.00012320532641838186, nan]"
4,1.0,1.0,1.0,1.0,"[0.5253747094739183, 0.016854412839272838]",1.008001,"[0.00012320532641838186, nan]"
